# Prototyping web apps with xlwings

The following app gives an idea of how xlwings can be used to prototype web apps:

https://github.com/ZoomerAnalytics/simulation-demo

# Deployment

## Plain vanilla: Excel + Python files

You can, e.g., send around a zip file containing both the Excel and the Python file. But this means that the receipient always needs to keep the files together.

## Python package

By default, the xlwings VBA settings add the path of the Excel file to the `PYTHONPATH` which means that you are able to do `RunPython(import mymodule)`, if `mymodule` is in the same directory as the Excel file. However, xlwings also finds modules/packages anywhere else following Python's rules of the **module search path**.

This mainly means that the directory `site-packages`, which is Python's default location for 3rd party modules/packages, is also accessible. So when distributing your software to end-users, you may prefer to create a python package for distribution. Advantages you get are:

* You can `pip/conda install`
* You can put a version string on your package
* The code gets out of sight of the end-users, leaving them with an experience no different from a file with standard VBA only: They can move the Excel file wherever they want without having to care about moving along a second file.

## Let's build a Python package

* let's use the hello world example from the quickstart project 
* add a version string to the source in `hello.py` after the import section: `__version__ = '1.0.0'`  
* create the following minimalistic `setup.py` file next to the `hello.py` module:

In [ ]:
import os
import re
from setuptools import setup

# Take the version number from hello.py
with open(os.path.join(os.path.dirname(__file__), 'hello.py')) as f:
    version = re.compile(r".*__version__ = '(.*?)'", re.S).match(f.read()).group(1)

setup(
    name="hello",
    version=version,
    py_modules=['hello'],
)

Now let's run:

`python setup.py sdist`

You will get some warnings as the setup file is actually too minimalistic if you wanted to register it on PyPI. However, to use it locally, it will be just fine.

Now, make sure to change into a different directory from your source folder, i.e. out of `fibonacci` directory and run:

`pip install path/to/hello-1.0.0.zip` 

Note that `hello-1.0.0.zip/tar.gz` is located inside the `dist` folder.

Check the installation:

In [ ]:
import hello
hello.__version__

Give it a try, you can now move around the `hello.xlsm` and can forget about the python source file.

When you're done, you may want to uninstall the module again with:

`pip uninstall hello`

## Python package servers

Possibilities to distribute your python package include:

* direct distribution via shared drive, email, intranet/internet, e.g.: `pip install \\drive\mypackage.zip`
* public distribution via PyPI/company-internal PyPI server
* paid hosted services for private packages like:
    * http://anaconda.org/ (conda package)
    * https://gemfury.com/ (pip package)
* distribution as self-installing package: `python setup.py bdist_wininst` 
* you can also pip install directly from a git repository (in- or external), targeting specific version tags or commits, e.g.:  
`pip install git+https://git.myproject.org/MyProject.git`

## Get rid of the xlwings add-in dependency

Use the quickstart command with the `--standalone` option:

`xlwings quickstart hello --standalone`

This includes the content of the add-in as standard VBA module.

## RunFrozenPython

Sometimes you might find it beneficial to turn your program into an executable so you can distribute it without requiring an installation of Python. There are different "freezers" available (py2exe, cx_Freeze) but we're using pyinstaller.

`pip install pyinstaller`

First, let's create a new standalone project:

`xlwings quickstart frozen --standalone`

add this code to `frozen.py` to create an entry point for pyinstaller:

In [ ]:
if __name__ == "__main__":
    hello_xlwings()

then, compile the executable:

In [ ]:
pyinstaller frozen.py

This will create a config file called hello.spec next to your `frozen.py` file. Since pyinstaller is including all optional dependencies, too, we need to explicitly tell it to not include them. Open `frozen.spec` and under `Analysis`, list the following for `excludes`:

In [ ]:
 excludes=['scipy', 'numpy', 'email', 'xml', 'pandas', 'Tkinter', 'Tkconstants', 'pydoc',
           'tcl', 'tk', 'matplotlib', 'PIL', 'nose', 'setuptools', 'xlrd', 'xlwt', 'PyQt5',
           'markdown', 'IPython', 'docutils']

After changing the spec file, you need to re-run pyinstaller using this command (**NOTE**: `.spec` instead of `.py` file!):

`pyinstaller frozen.spec`

Next, you need to use this function in Excel to call the frozen version:

In [ ]:
Sub SampleCall()
    RunFrozenPython ThisWorkbook.Path & "\dist\frozen\frozen.exe"
End Sub

As final step, you may want to package together everything with an installer like Inno Setup or NSIS. In that case, you might also want to place the executable in a standard writable location like `%APPDATA%` or `Program Files`.

<div class="alert alert-info">

**Note**: User Defined Functions are currently not supported with frozen executables, only RunPython functions.

</div>